In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

Data Generation
===

In [ ]:
np.random.seed(10)
p, q = (np.random.rand(i, 2) for i in (4, 5))
p_big, q_big = (np.random.rand(i, 80) for i in (100, 120))

print(p, "\n\n", q)

Solution
===

In [ ]:
def naive(p, q):
    result = np.zeros((p.shape[0], q.shape[0]))
    for i in range(p.shape[0]):
        for j in range(q.shape[0]):
            tmp = 0
            for k in range(p.shape[1]):
                tmp += (p[i, k] - q[j, k]) ** 2
            result[i, j] = tmp
    return np.sqrt(result)

In [ ]:
def naive_2(p, q):
    result = np.zeros((p.shape[0], q.shape[0]))
    for i in range(p.shape[0]):
        for j in range(q.shape[0]):
            result[i, j] = np.sum((p[i] - q[j]) ** 2)
    return np.sqrt(result)

### Use matching indices

Instead of iterating through indices, one can use them directly to parallelize the operations with Numpy.

In [ ]:
rows, cols = np.indices((p.shape[0], q.shape[0]))
print(rows, end="\n\n")
print(cols)

In [ ]:
print(p[rows.ravel()], end="\n\n")
print(q[cols.ravel()])

In [ ]:
def with_indices(p, q):
    rows, cols = np.indices((p.shape[0], q.shape[0]))
    distances = np.sqrt(np.sum((p[rows.ravel(), :] - q[cols.ravel(), :]) ** 2, axis=1))
    return distances.reshape((p.shape[0], q.shape[0]))

In [ ]:
def with_indices_2(p, q):
    rows, cols = np.indices((p.shape[0], q.shape[0]))
    distances = np.sqrt(np.sum((p[rows, :] - q[cols, :]) ** 2, axis=2))
    return distances

### Use a library

`scipy` is the equivalent of matlab toolboxes and have a lot to offer. Actually the pairwise computation is part of the library through the `spatial` module.

In [ ]:
from scipy.spatial.distance import cdist


def scipy_version(p, q):
    return cdist(p, q)

### Numpy Magic

In [ ]:
def tensor_broadcasting(p, q):
    return np.sqrt(np.sum((p[:, np.newaxis, :] - q[np.newaxis, :, :]) ** 2, axis=2))

# Compare methods

In [ ]:
methods = [
    naive,
    naive_2,
    with_indices,
    with_indices_2,
    scipy_version,
    tensor_broadcasting,
]
timers = []
for f in methods:
    r = %timeit -o f(p_big, q_big)
    timers.append(r)

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(
    np.arange(len(methods)), [r.best * 1000 for r in timers], log=False
)  # Set log to True for logarithmic scale
plt.xticks(np.arange(len(methods)) + 0.2, [f.__name__ for f in methods], rotation=30)
plt.xlabel("Method")
plt.ylabel("Time (ms)")
plt.show()